In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import seaborn as sns
from itertools import combinations
from pandas import DataFrame
import sklearn
import scikitplot as skplt
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
svm=LinearSVC()
kn=KNeighborsClassifier()
rf=RandomForestClassifier()
gnb=GaussianNB()

In [3]:
import matplotlib as mt
axes= {"linewidth" : 3}
mt.rc("axes", **axes)
font={"weight": 'bold', "family": "Times new roman", "size": 20}
mt.rc("font", **font)

In [4]:
dataset=pd.read_csv("Partial Data Points.csv")
print(dataset.columns)

Index(['ZrB2', 'HfB2', 'SiC', 'TaB2', 'Ta', 'C', 'ZrC', 'ZrSi2', 'MoSi2',
       'B4C', 'Si3N4', 'TaSi2', 'WC', 'WB', 'MoSi', 'HfC', 'BN', 'TaC',
       'Ta5Si3', 'LaB6', 'HfSi2', 'ZrO2', 'Sinter method', 'Sinter temp',
       'Sintering Pressure (Mpa)', 'Sintering Time (min)', 'Relative Density',
       'GS in microns', 'Temp', 'Oxidation Test', 'Time@Temp (s)',
       'Time@Temp (min)', 'Atmospheric Prerssure', 'Oxygen Partial Pressure',
       'Atmospheric Conditions', 'Flow Rate (cm3/min)', 'Oxide Thickness (um)',
       'Mass Gain (mg/cm2)'],
      dtype='object')


In [5]:
filled_cells_below_each_column = {}

for column in dataset.columns:
    filled_cells_below_each_column[column] = dataset[column].notnull().sum()

# Print the result
for column, count in filled_cells_below_each_column.items():
    print(f"{column}- {count}")    

ZrB2- 528
HfB2- 345
SiC- 508
TaB2- 1
Ta- 1
C- 3
ZrC- 128
ZrSi2- 65
MoSi2- 146
B4C- 65
Si3N4- 67
TaSi2- 135
WC- 9
WB- 6
MoSi- 65
HfC- 130
BN- 2
TaC- 129
Ta5Si3- 1
LaB6- 131
HfSi2- 1
ZrO2- 4
Sinter method- 501
Sinter temp- 489
Sintering Pressure (Mpa)- 467
Sintering Time (min)- 485
Relative Density- 456
GS in microns- 297
Temp- 528
Oxidation Test- 386
Time@Temp (s)- 128
Time@Temp (min)- 528
Atmospheric Prerssure- 12
Oxygen Partial Pressure- 12
Atmospheric Conditions- 238
Flow Rate (cm3/min)- 215
Oxide Thickness (um)- 463
Mass Gain (mg/cm2)- 71


In [6]:
oxide_thickness_df = dataset[dataset['Oxide Thickness (um)'].notnull()]
print(oxide_thickness_df)

     ZrB2  HfB2   SiC  TaB2  Ta   C    ZrC  ZrSi2  MoSi2  B4C  ...     Temp  \
0     0.0   0.0   0.0   NaN NaN NaN   80.0    0.0    0.0  0.0  ...   800.00   
1     0.0   0.0   0.0   NaN NaN NaN  100.0    0.0    0.0  0.0  ...   800.00   
2     0.0   0.0   0.0   NaN NaN NaN    0.0    0.0    0.0  0.0  ...  2400.00   
3     0.0   0.0   0.0   NaN NaN NaN   80.0    0.0   20.0  0.0  ...  1526.85   
4     0.0   0.0   0.0   NaN NaN NaN   80.0    0.0   20.0  0.0  ...  1526.85   
..    ...   ...   ...   ...  ..  ..    ...    ...    ...  ...  ...      ...   
517  80.0   NaN  20.0   NaN NaN NaN    NaN    NaN    NaN  NaN  ...  1800.00   
519  50.0   NaN  30.0   NaN NaN NaN    NaN    NaN    NaN  NaN  ...  1800.00   
525  70.0   NaN  30.0   NaN NaN NaN    NaN    NaN    NaN  NaN  ...  2000.00   
526  70.0   NaN  30.0   NaN NaN NaN    NaN    NaN    NaN  NaN  ...  2000.00   
527  70.0   NaN  30.0   NaN NaN NaN    NaN    NaN    NaN  NaN  ...  2000.00   

     Oxidation Test  Time@Temp (s)  Time@Temp (min)

In [7]:
filled_cells_below_each_column = {}

for column in oxide_thickness_df.columns:
    filled_cells_below_each_column[column] = oxide_thickness_df[column].notnull().sum()

# Print the result
for column, count in filled_cells_below_each_column.items():
    print(f"{column}- {count}") 

ZrB2- 463
HfB2- 337
SiC- 461
TaB2- 0
Ta- 1
C- 2
ZrC- 128
ZrSi2- 65
MoSi2- 128
B4C- 65
Si3N4- 66
TaSi2- 131
WC- 7
WB- 6
MoSi- 65
HfC- 128
BN- 0
TaC- 128
Ta5Si3- 1
LaB6- 131
HfSi2- 0
ZrO2- 0
Sinter method- 436
Sinter temp- 427
Sintering Pressure (Mpa)- 405
Sintering Time (min)- 423
Relative Density- 395
GS in microns- 291
Temp- 463
Oxidation Test- 321
Time@Temp (s)- 128
Time@Temp (min)- 463
Atmospheric Prerssure- 12
Oxygen Partial Pressure- 12
Atmospheric Conditions- 177
Flow Rate (cm3/min)- 154
Oxide Thickness (um)- 463
Mass Gain (mg/cm2)- 6


In [8]:
columns_to_remove = ['Mass Gain (mg/cm2)', 'Flow Rate (cm3/min)', 'Atmospheric Conditions', 
                     'Oxygen Partial Pressure', 'Atmospheric Prerssure', 'Time@Temp (s)', 'GS in microns']

# Remove specified columns
cleaned_oxide_df = oxide_thickness_df.drop(columns=columns_to_remove)
print(cleaned_oxide_df)

     ZrB2  HfB2   SiC  TaB2  Ta   C    ZrC  ZrSi2  MoSi2  B4C  ...  ZrO2  \
0     0.0   0.0   0.0   NaN NaN NaN   80.0    0.0    0.0  0.0  ...   NaN   
1     0.0   0.0   0.0   NaN NaN NaN  100.0    0.0    0.0  0.0  ...   NaN   
2     0.0   0.0   0.0   NaN NaN NaN    0.0    0.0    0.0  0.0  ...   NaN   
3     0.0   0.0   0.0   NaN NaN NaN   80.0    0.0   20.0  0.0  ...   NaN   
4     0.0   0.0   0.0   NaN NaN NaN   80.0    0.0   20.0  0.0  ...   NaN   
..    ...   ...   ...   ...  ..  ..    ...    ...    ...  ...  ...   ...   
517  80.0   NaN  20.0   NaN NaN NaN    NaN    NaN    NaN  NaN  ...   NaN   
519  50.0   NaN  30.0   NaN NaN NaN    NaN    NaN    NaN  NaN  ...   NaN   
525  70.0   NaN  30.0   NaN NaN NaN    NaN    NaN    NaN  NaN  ...   NaN   
526  70.0   NaN  30.0   NaN NaN NaN    NaN    NaN    NaN  NaN  ...   NaN   
527  70.0   NaN  30.0   NaN NaN NaN    NaN    NaN    NaN  NaN  ...   NaN   

              Sinter method  Sinter temp  Sintering Pressure (Mpa)  \
0    Spark Plasme

In [9]:
filled_cells_below_each_column = {}

for column in cleaned_oxide_df.columns:
    filled_cells_below_each_column[column] = cleaned_oxide_df[column].notnull().sum()

# Print the result
for column, count in filled_cells_below_each_column.items():
    print(f"{column}- {count}") 

ZrB2- 463
HfB2- 337
SiC- 461
TaB2- 0
Ta- 1
C- 2
ZrC- 128
ZrSi2- 65
MoSi2- 128
B4C- 65
Si3N4- 66
TaSi2- 131
WC- 7
WB- 6
MoSi- 65
HfC- 128
BN- 0
TaC- 128
Ta5Si3- 1
LaB6- 131
HfSi2- 0
ZrO2- 0
Sinter method- 436
Sinter temp- 427
Sintering Pressure (Mpa)- 405
Sintering Time (min)- 423
Relative Density- 395
Temp- 463
Oxidation Test- 321
Time@Temp (min)- 463
Oxide Thickness (um)- 463


In [10]:
siC_present_oxide_df = cleaned_oxide_df[cleaned_oxide_df['SiC'].notnull()]
print(siC_present_oxide_df)

     ZrB2  HfB2   SiC  TaB2  Ta   C    ZrC  ZrSi2  MoSi2  B4C  ...  ZrO2  \
0     0.0   0.0   0.0   NaN NaN NaN   80.0    0.0    0.0  0.0  ...   NaN   
1     0.0   0.0   0.0   NaN NaN NaN  100.0    0.0    0.0  0.0  ...   NaN   
2     0.0   0.0   0.0   NaN NaN NaN    0.0    0.0    0.0  0.0  ...   NaN   
3     0.0   0.0   0.0   NaN NaN NaN   80.0    0.0   20.0  0.0  ...   NaN   
4     0.0   0.0   0.0   NaN NaN NaN   80.0    0.0   20.0  0.0  ...   NaN   
..    ...   ...   ...   ...  ..  ..    ...    ...    ...  ...  ...   ...   
517  80.0   NaN  20.0   NaN NaN NaN    NaN    NaN    NaN  NaN  ...   NaN   
519  50.0   NaN  30.0   NaN NaN NaN    NaN    NaN    NaN  NaN  ...   NaN   
525  70.0   NaN  30.0   NaN NaN NaN    NaN    NaN    NaN  NaN  ...   NaN   
526  70.0   NaN  30.0   NaN NaN NaN    NaN    NaN    NaN  NaN  ...   NaN   
527  70.0   NaN  30.0   NaN NaN NaN    NaN    NaN    NaN  NaN  ...   NaN   

              Sinter method  Sinter temp  Sintering Pressure (Mpa)  \
0    Spark Plasme

In [11]:
siC_present_oxide_df.iloc[:, :22] = siC_present_oxide_df.iloc[:, :22].fillna(0)
print(siC_present_oxide_df)

     ZrB2  HfB2   SiC  TaB2   Ta    C    ZrC  ZrSi2  MoSi2  B4C  ...  ZrO2  \
0     0.0   0.0   0.0   0.0  0.0  0.0   80.0    0.0    0.0  0.0  ...   0.0   
1     0.0   0.0   0.0   0.0  0.0  0.0  100.0    0.0    0.0  0.0  ...   0.0   
2     0.0   0.0   0.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
3     0.0   0.0   0.0   0.0  0.0  0.0   80.0    0.0   20.0  0.0  ...   0.0   
4     0.0   0.0   0.0   0.0  0.0  0.0   80.0    0.0   20.0  0.0  ...   0.0   
..    ...   ...   ...   ...  ...  ...    ...    ...    ...  ...  ...   ...   
517  80.0   0.0  20.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
519  50.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
525  70.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
526  70.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
527  70.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   

              Sinter method  Sinter temp  Sintering Pressure (M

In [12]:
siC_present_oxide_oxidation_test_df = siC_present_oxide_df[siC_present_oxide_df['Oxidation Test'].notnull()]
print(siC_present_oxide_oxidation_test_df)

     ZrB2  HfB2   SiC  TaB2   Ta    C    ZrC  ZrSi2  MoSi2  B4C  ...  ZrO2  \
0     0.0   0.0   0.0   0.0  0.0  0.0   80.0    0.0    0.0  0.0  ...   0.0   
1     0.0   0.0   0.0   0.0  0.0  0.0  100.0    0.0    0.0  0.0  ...   0.0   
2     0.0   0.0   0.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
3     0.0   0.0   0.0   0.0  0.0  0.0   80.0    0.0   20.0  0.0  ...   0.0   
4     0.0   0.0   0.0   0.0  0.0  0.0   80.0    0.0   20.0  0.0  ...   0.0   
..    ...   ...   ...   ...  ...  ...    ...    ...    ...  ...  ...   ...   
517  80.0   0.0  20.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
519  50.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
525  70.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
526  70.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
527  70.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   

              Sinter method  Sinter temp  Sintering Pressure (M

In [13]:
filled_cells_below_each_column = {}

for column in siC_present_oxide_oxidation_test_df.columns:
    filled_cells_below_each_column[column] = siC_present_oxide_oxidation_test_df[column].notnull().sum()

# Print the result
for column, count in filled_cells_below_each_column.items():
    print(f"{column}- {count}") 

ZrB2- 319
HfB2- 319
SiC- 319
TaB2- 319
Ta- 319
C- 319
ZrC- 319
ZrSi2- 319
MoSi2- 319
B4C- 319
Si3N4- 319
TaSi2- 319
WC- 319
WB- 319
MoSi- 319
HfC- 319
BN- 319
TaC- 319
Ta5Si3- 319
LaB6- 319
HfSi2- 319
ZrO2- 319
Sinter method- 301
Sinter temp- 296
Sintering Pressure (Mpa)- 284
Sintering Time (min)- 292
Relative Density- 275
Temp- 319
Oxidation Test- 319
Time@Temp (min)- 319
Oxide Thickness (um)- 319


In [14]:
siC_present_oxide_relative_df = siC_present_oxide_oxidation_test_df[siC_present_oxide_oxidation_test_df['Relative Density'].notnull()]
print(siC_present_oxide_relative_df)

     ZrB2  HfB2   SiC  TaB2   Ta    C    ZrC  ZrSi2  MoSi2  B4C  ...  ZrO2  \
0     0.0   0.0   0.0   0.0  0.0  0.0   80.0    0.0    0.0  0.0  ...   0.0   
1     0.0   0.0   0.0   0.0  0.0  0.0  100.0    0.0    0.0  0.0  ...   0.0   
2     0.0   0.0   0.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
3     0.0   0.0   0.0   0.0  0.0  0.0   80.0    0.0   20.0  0.0  ...   0.0   
4     0.0   0.0   0.0   0.0  0.0  0.0   80.0    0.0   20.0  0.0  ...   0.0   
..    ...   ...   ...   ...  ...  ...    ...    ...    ...  ...  ...   ...   
517  80.0   0.0  20.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
519  50.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
525  70.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
526  70.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
527  70.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   

              Sinter method  Sinter temp  Sintering Pressure (M

In [15]:
filled_cells_below_each_column = {}

for column in siC_present_oxide_relative_df.columns:
    filled_cells_below_each_column[column] = siC_present_oxide_relative_df[column].notnull().sum()

# Print the result
for column, count in filled_cells_below_each_column.items():
    print(f"{column}- {count}") 

ZrB2- 275
HfB2- 275
SiC- 275
TaB2- 275
Ta- 275
C- 275
ZrC- 275
ZrSi2- 275
MoSi2- 275
B4C- 275
Si3N4- 275
TaSi2- 275
WC- 275
WB- 275
MoSi- 275
HfC- 275
BN- 275
TaC- 275
Ta5Si3- 275
LaB6- 275
HfSi2- 275
ZrO2- 275
Sinter method- 275
Sinter temp- 271
Sintering Pressure (Mpa)- 263
Sintering Time (min)- 271
Relative Density- 275
Temp- 275
Oxidation Test- 275
Time@Temp (min)- 275
Oxide Thickness (um)- 275


In [16]:
# Assuming df is your DataFrame
cleaned_df = siC_present_oxide_relative_df.dropna()

# If you want to remove rows with NaN values only in specific columns, you can specify them
# For example, to remove rows with NaN values in any of the columns you mentioned:
cols_to_check = ['ZrB2', 'HfB2', 'SiC', 'Ta', 'C', 'ZrC', 'ZrSi2', 'MoSi2',
                 'B4C', 'Si3N4', 'TaSi2', 'WC', 'WB', 'MoSi', 'HfC', 'TaC',
                 'Ta5Si3', 'LaB6', 'Sinter method', 'Sinter temp',
                 'Sintering Pressure (Mpa)', 'Sintering Time (min)', 'Relative Density',
                 'Temp', 'Oxidation Test', 'Time@Temp (min)', 'Oxide Thickness (um)']
cleaned_oxide_thickness_final_df = siC_present_oxide_relative_df.dropna(subset=cols_to_check)

# This will remove rows with NaN values in any of the specified columns
print(cleaned_oxide_thickness_final_df)

     ZrB2  HfB2   SiC  TaB2   Ta    C    ZrC  ZrSi2  MoSi2  B4C  ...  ZrO2  \
0     0.0   0.0   0.0   0.0  0.0  0.0   80.0    0.0    0.0  0.0  ...   0.0   
1     0.0   0.0   0.0   0.0  0.0  0.0  100.0    0.0    0.0  0.0  ...   0.0   
2     0.0   0.0   0.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
3     0.0   0.0   0.0   0.0  0.0  0.0   80.0    0.0   20.0  0.0  ...   0.0   
4     0.0   0.0   0.0   0.0  0.0  0.0   80.0    0.0   20.0  0.0  ...   0.0   
..    ...   ...   ...   ...  ...  ...    ...    ...    ...  ...  ...   ...   
517  80.0   0.0  20.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
519  50.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
525  70.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
526  70.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
527  70.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   

              Sinter method  Sinter temp  Sintering Pressure (M

In [17]:
cleaned_oxide_thickness_final_df = cleaned_oxide_thickness_final_df.reset_index(drop=True)
print(cleaned_oxide_thickness_final_df)

     ZrB2  HfB2   SiC  TaB2   Ta    C    ZrC  ZrSi2  MoSi2  B4C  ...  ZrO2  \
0     0.0   0.0   0.0   0.0  0.0  0.0   80.0    0.0    0.0  0.0  ...   0.0   
1     0.0   0.0   0.0   0.0  0.0  0.0  100.0    0.0    0.0  0.0  ...   0.0   
2     0.0   0.0   0.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
3     0.0   0.0   0.0   0.0  0.0  0.0   80.0    0.0   20.0  0.0  ...   0.0   
4     0.0   0.0   0.0   0.0  0.0  0.0   80.0    0.0   20.0  0.0  ...   0.0   
..    ...   ...   ...   ...  ...  ...    ...    ...    ...  ...  ...   ...   
258  80.0   0.0  20.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
259  50.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
260  70.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
261  70.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
262  70.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   

              Sinter method  Sinter temp  Sintering Pressure (M

In [18]:
rows_to_remove = [262,261,260,259,258,257,224,223,222,221,220,219,88,87,86,85,84,83,82,81,80,79,78,76,77,75,74,27,26,25,24,23,22,21,20,19,18,17,16,15,14,13]
cleaned_oxide_thickness_final_df = cleaned_oxide_thickness_final_df.drop(rows_to_remove)
print(cleaned_oxide_thickness_final_df)

     ZrB2  HfB2   SiC  TaB2   Ta    C    ZrC  ZrSi2  MoSi2  B4C  ...  ZrO2  \
0     0.0   0.0   0.0   0.0  0.0  0.0   80.0    0.0    0.0  0.0  ...   0.0   
1     0.0   0.0   0.0   0.0  0.0  0.0  100.0    0.0    0.0  0.0  ...   0.0   
2     0.0   0.0   0.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
3     0.0   0.0   0.0   0.0  0.0  0.0   80.0    0.0   20.0  0.0  ...   0.0   
4     0.0   0.0   0.0   0.0  0.0  0.0   80.0    0.0   20.0  0.0  ...   0.0   
..    ...   ...   ...   ...  ...  ...    ...    ...    ...  ...  ...   ...   
252   0.0  82.0  15.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
253  77.6   0.0  22.4   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
254  70.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
255  70.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
256  70.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   

              Sinter method  Sinter temp  Sintering Pressure (M

In [19]:
cleaned_oxide_thickness_final_df['Oxidation Test'] = cleaned_oxide_thickness_final_df['Oxidation Test'].replace({
    'Furnace': 1,
     'furnace':1,
     'Plasma Jet':2,
      'oxyacetylene torch':3,
      'Arc-Jet':4})
#print(filtered_df)
cleaned_oxide_thickness_final_df['Sinter method'] = cleaned_oxide_thickness_final_df['Sinter method'].replace({
    'Hot pressed': 1,
    'Spark Plasma Sintering': 2,
    'Spark Plasme Sintering': 2,
    'hot pressed': 1,
    'Pressureless sintering': 3,
    'Pressureless Sintering' :3,
    'Pressurless Sintering' : 3
})
print(cleaned_oxide_thickness_final_df)

     ZrB2  HfB2   SiC  TaB2   Ta    C    ZrC  ZrSi2  MoSi2  B4C  ...  ZrO2  \
0     0.0   0.0   0.0   0.0  0.0  0.0   80.0    0.0    0.0  0.0  ...   0.0   
1     0.0   0.0   0.0   0.0  0.0  0.0  100.0    0.0    0.0  0.0  ...   0.0   
2     0.0   0.0   0.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
3     0.0   0.0   0.0   0.0  0.0  0.0   80.0    0.0   20.0  0.0  ...   0.0   
4     0.0   0.0   0.0   0.0  0.0  0.0   80.0    0.0   20.0  0.0  ...   0.0   
..    ...   ...   ...   ...  ...  ...    ...    ...    ...  ...  ...   ...   
252   0.0  82.0  15.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
253  77.6   0.0  22.4   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
254  70.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
255  70.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   
256  70.0   0.0  30.0   0.0  0.0  0.0    0.0    0.0    0.0  0.0  ...   0.0   

     Sinter method  Sinter temp  Sintering Pressure (Mpa)  \
0 

In [20]:
cleaned_oxide_thickness_final_df.to_csv('oxide_thickness_SiC_sorted.csv', index=False)